In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
from tqdm import tqdm
import sys
sys.path.append("../src")
from event_selections import *

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
def drop_columns(df):
    #kept_columns=['layer', 'strip', 'pedestal', 'pf_event', 'adc_sum_end0', 'adc_sum_end1', 'end', 'mpv', 'std_dev']
    #df=df[df.columns.intersection(kept_columns)]
    return df

def import_data(calibration_folder, data_folder, run_n):
    pedestals=drop_columns(pd.read_csv(calibration_folder+"/pedestals_MIP.csv", sep=','))
    mips=drop_columns(pd.read_csv(calibration_folder+"/mip.csv", sep=','))
    run=drop_columns(pd.read_csv(data_folder+"/run_"+str(run_n)+"_pulse.csv", sep=','))
    # run=run.astype({"adc_sum_end0":float,"adc_sum_end1":float}) 
    return pedestals,mips,run

In [4]:
def drop_empty(df):
    data=[]

    layers=np.arange(1,20)
    strips=np.arange(0,12)
    
    for layer in layers:
        for strip in strips:
            el=choose_bar(df,layer,strip)
            peds=choose_bar(pedestal_df,layer,strip)
            # for now, we don't deal with conversion to MeV
            #mip=choose_bar(mips, layer, strip)

            if not peds.empty: # need to check whether the strip exists 
                
                # check whetehr signal is large enough
                el=el[el["adc_sum_end0"]>(peds.iloc[0,3]+5*peds.iloc[0,5])] 
                el=el[el["adc_sum_end1"]>(peds.iloc[1,3]+5*peds.iloc[1,5])] 

                # subtract from sum
                el.loc[:,"adc_sum_end0"]-=peds.iloc[0,3]
                el.loc[:,"adc_sum_end1"]-=peds.iloc[1,3]
                
                # subtract from max
                el.loc[:,"adc_max_end0"]-=peds.iloc[0,6]
                el.loc[:,"adc_max_end1"]-=peds.iloc[1,6]

                # subtract from mean
                el.loc[:,"adc_mean_end0"]-=peds.iloc[0,6]
                el.loc[:,"adc_mean_end1"]-=peds.iloc[1,6]
                
                # miniped=peds.iloc[0,-2]/8
                # miniped1=peds.iloc[1,-2]/8

                # subtract from timestamps
                el.loc[:,"adc_0_end0"]-=peds.iloc[0,6]
                el.loc[:,"adc_1_end0"]-=peds.iloc[0,6]
                el.loc[:,"adc_2_end0"]-=peds.iloc[0,6]
                el.loc[:,"adc_3_end0"]-=peds.iloc[0,6]
                el.loc[:,"adc_4_end0"]-=peds.iloc[0,6]
                el.loc[:,"adc_5_end0"]-=peds.iloc[0,6]
                el.loc[:,"adc_6_end0"]-=peds.iloc[0,6]
                el.loc[:,"adc_7_end0"]-=peds.iloc[0,6]

                el.loc[:,"adc_0_end1"]-=peds.iloc[1,6]
                el.loc[:,"adc_1_end1"]-=peds.iloc[1,6]
                el.loc[:,"adc_2_end1"]-=peds.iloc[1,6]
                el.loc[:,"adc_3_end1"]-=peds.iloc[1,6]
                el.loc[:,"adc_4_end1"]-=peds.iloc[1,6]
                el.loc[:,"adc_5_end1"]-=peds.iloc[1,6]
                el.loc[:,"adc_6_end1"]-=peds.iloc[1,6]
                el.loc[:,"adc_7_end1"]-=peds.iloc[1,6]
                
                data.extend(el.values.tolist())
                
    names=list(df.columns)

    df_s=pd.DataFrame(data, columns=names)
    
    return df_s

In [5]:
c_folder="../../data/calibration"
d_folder="../../data/runData"
r_n=307
pulsed=True
p_folder="../../plots/run_"+str(r_n)

In [6]:
pedestal_df, mip_df, run_df=import_data(c_folder, d_folder,r_n)

In [7]:
# first, we drop the TOA columns. TOA has proven to be unreliable in telling whether there is an incoming pulse or not
run_df=run_df.drop(columns=['toa_end0', 'toa_end1'])

In [8]:
# next, we drop every bar that is "empty", meaning that the signal does not go above pedestal + 5 sigma
# we also subtract pedestals in this step
run_df=drop_empty(run_df)

In [9]:
def isolate_pulse_data(row):
    end0 = []
    end1 = []

    for j in range(8):
        adc_end0_col = f'adc_{j}_end0'
        adc_end1_col = f'adc_{j}_end1'
        
        end0_val = row[adc_end0_col]
        end1_val = row[adc_end1_col]
        
        end0.append(end0_val)
        end1.append(end1_val)

    end0 = np.array(end0)
    end1 = np.array(end1)
    return end0,end1

In [10]:
def plot_pulse(end0, end1, layer, bar, event):
    plt.scatter(np.arange(8), end0, label="end0", c="cyan", alpha=0.5)
    plt.scatter(np.arange(8), end1, label="end1", c="magenta", alpha=0.5)
    plt.xlabel("Timestep")
    plt.ylabel("ADC")
    plt.title("Layer: "+str(layer)+" Bar: "+str(bar)+" Event: "+str(event))
    plt.legend()
    plt.show()
    return

In [11]:
def does_pulse_dip(end, p_df, layer, bar, which):
    # get standard deviation of pedestal. I believe this is equivalent to the uncertainty of ani measurement
    # or at least it is a way to approximate that
    
    end_std=p_df[(p_df.layer==layer) & (p_df.strip==bar) & (p_df.end==which)].iloc[0]["pedestal_per_time_sample_std_dev"]

    # I think this doesn't catch dips that have a perfectly flat bottom
    end_diff=np.diff(end)
    end_sign=np.sign(end_diff)
    end_sdiff=np.diff(end_sign)

    dip_loc=np.isin(end_sdiff,2)
    dip_loc=np.append(dip_loc,False)
    dip_value=abs(end_diff[dip_loc])

    if dip_value.size==1 and dip_value>5*end_std:
        return 1
    elif dip_value.size>1 and dip_value.any()>5*end_std:
        print("Something went wrong.")
        return 1
        
    # there could be some failsafe here but I cannot be bothered rn
    return 0
    

In [12]:
def late_filter(end):
    tolerance = 0.2
    if end[-1] >= (end[0] + max(end) * tolerance) or (end[-1] <= end[0] - max(end) * tolerance):

        tolerance = 0.1
        if end[1] <= (end[0] + end[0] * tolerance) and end[1] >= (end[0] - end[0] * tolerance):
            if end[2] <= (end[0] + end[0] * tolerance) and end[2] >= (end[0] - end[0] * tolerance):
                return 1

    return 0

In [13]:
def categorize_pulses(row, p_df):
    out_line=[row.name]
    
    # isolating datapoints from row
    end0, end1=isolate_pulse_data(row)

    # preparing to filter
    layer=row["layer"]    # used as variables to avoid frequent lookup
    bar=row["strip"]
    event=row["pf_event"]

    out_line.append(layer)
    out_line.append(bar)
    out_line.append(event)

    # spike and dip filter
    
    dip_end0=does_pulse_dip(end0, p_df, layer, bar, 0)
    dip_end1=does_pulse_dip(end1, p_df, layer, bar, 1)

    if dip_end0==1 and dip_end1==1:
        #print("Wavy pulse")
        out_line.append(1)
        out_line.append("WAVE")
    elif dip_end0==1 and dip_end1==0:
        #print("Spike in end0")
        out_line.append(1)
        out_line.append("SPIKE_0")
    elif dip_end0==0 and dip_end1==1:
        #print("Spike in end0")
        out_line.append(1)
        out_line.append("SPIKE_1")
    else:
        #print("No dipping")

        late_end0=late_filter(end0)
        late_end1=late_filter(end1)
    
        if late_end0==1 or late_end1==1:
            #print("Late pulse")
            out_line.append(1)
            out_line.append("LATE")
        else:
            #print("Good pulse")
            out_line.append(0)
            out_line.append("NaN")

    
    
    #plot_pulse(end0, end1, layer, bar, event)
    return out_line

In [14]:
tqdm.pandas()
problems=run_df.progress_apply(categorize_pulses, args=(pedestal_df,), axis=1).values.tolist()

100%|██████████████████████████████████| 172271/172271 [04:36<00:00, 623.69it/s]


In [15]:
pulse_df=pd.DataFrame(problems, columns=["index","layer", "bar", "event", "has_problem", "problem_type"])
pulse_df.set_index("index", inplace=True)

In [16]:
pulse_df.head()

,layer,bar,event,has_problem,problem_type
index,,,,,
0,1.0,0.0,36.0,0,NaN
1,1.0,0.0,91.0,0,NaN
2,1.0,0.0,175.0,0,NaN
3,1.0,0.0,277.0,0,NaN
4,1.0,0.0,368.0,0,NaN


In [22]:
# pulse_df.to_csv(d_folder+"/run_"+str(r_n)+"_pulse_problems.csv")